This is intended to be a scratchpad for measures of between-subject SST inhibition.

This is based on SST_inhibition_basic_tests.ipynb.



### SST as a measure of inhibition

Our subject-level behavioral measure is SSRT. But it seems like Post-Error slowing is another important measure. So what about subject-level?



 - Similarity of successful no-go trials to successful go trials
 - Similarity of successful no-go trials to failed no-go trials
 
 
 
 - Similarity of NoGo trials

For each measure we want to:

1. validate it works by showing whether we can do within-run LOOCV to identify nogo and go trials, on average, across the subject group
2. Reduce to a numerical value for each subject for between-subject comparison
3. Compare to ohter measures of inhibition
4. predict

We're going to do a subject-level analysis.

To regress at the subject level you would probably (?) take the average subject image and regress across subjects. This would be fairly straightforward. Then, you'd regress on:

 - various measures of TESQ-E
 - Other measures of inhibition? Bis11?
 - behavioral measures of the SST including mean session SSRT and PES

Some basic steps to do the subject-level stuff:
    
1) Do a single regressor across trials for each trial type
2) Across subjects, predict each of our measures of inhibition for each trial contrast we're interested in
3) Finish

That's really quite basic from an ML point of view but it might be all we're looking for here.

### Basic subject-level

First set up the data

In [4]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
import gc
import nibabel as nib

nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
test_train_df = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")


dataset_name = 'conditions'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_' + dataset_name + '_40subs.pkl'

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)
    
    


from nilearn.decoding import DecoderRegressor, Decoder

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/"
# HRF 2s



In [5]:

def condition_resp_trans_func(X):
    return(X.condition_label)


scaffold arguments for function in following paragraph

In [7]:
#set vars for the function

brain_data_filepath = brain_data_filepath
train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
subjs_to_use = 40
response_transform_func=condition_resp_trans_func
decoder = Decoder(scoring='accuracy',cv=LeaveOneGroupOut(),verbose=1)
mask=None

results_filepath=data_path + "SST/train_test_results_" + dataset_name + "_40subs.pkl"

### Functions to re-implement

The below are not run; they're set to "raw" only. See the below code.

`apply_loocv_and_save` loads the data and does do the final last-pass decoder. `cv_train_test_sets` does the meaty stuff. So, re-implementing that:

#### `apply_loocv_and_save` initial code

In [8]:
def decoderConstructor(*args, **kwargs):
    return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [9]:
cpus_to_use=40
subjs_to_use=40

In [19]:
# results_filepath,
# brain_data_filepath = '../data/Brain_Data_2sns_60subs.pkl',
# train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
# subjs_to_use = None, #set this to get a subset, otherwise use all of them.
# response_transform_func = None,
# mask=None,
# decoderConstructor=DecoderRegressor
# ):


preprocessed_data = load_and_preprocess(brain_data_filepath,train_test_markers_filepath,subjs_to_use,response_transform_func)

first_subs_nifti_Y = preprocessed_data['y']
first_subs_nifti = preprocessed_data['X']
first_subs_nifti_groups = preprocessed_data['groups']

print("starting LeaveOneOut")

#implicit CV done by Decoder's in-built CV algorithm. I cna't really turn this off.
cv_inner = GroupKFold(3)

decoder = decoderConstructor(
    standardize= True,cv=cv_inner,
    mask=mask,
    n_jobs=cpus_to_use)


#CV I'm writing myself
#in this design, we're actually dealing with groups
#we select group IDs and then grab the subjects
#so we don't need to use LeaveOneGroupOut
#the grouping is implicit
cv_outer = LeaveOneOut()




print("finished preprocessing")




checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 40 subjects
starting LeaveOneOut
finished preprocessing


#### cv_train_test_sets

In [36]:
trainset_y = first_subs_nifti_Y# = preprocessed_data['y']
trainset_X = first_subs_nifti# = preprocessed_data['X']
trainset_groups = first_subs_nifti_groups# = preprocessed_data['groups']


NameError: name 'first_subs_nifti_Y' is not defined

In [37]:
cpus_to_use=40
decoders = [decoder]
regressors=None
cv = None
param_grid=None
testset_X = None
testset_y = None
testset_groups=None
testset_groups = None

In [22]:
# def cv_train_test_sets(
#     trainset_X, trainset_y, trainset_groups,
#     decoders = None,
#     testset_X = None,testset_y = None, testset_groups = None,
#     param_grid=None,
#     cpus_to_use=-2,
#     cv = None,
#     regressors = None
#     ):
# """
# uses a division of 'trainset' and 'testset' to allow different values to be trained and tested 
# in KFold Cross Validation. All the values are used for training and testing, but we use different ones.
# This enables us to e.g., pass in aggregated images for training and separate images for testing.

# trainset_X: x values applicable for TRAINING
# trainset_y
# trainset_groups: group allocations for the trainset dataset
# testset_X: values grouped into averages for testing
# testset_y
# cv: a Grouped cross-validator
# group_list: name of the groups
# """

if regressors is not None:
    raise Exception("cv_train_test_sets 'regressors' argument is deprecated. Use 'decoders' instead. ")
if cv is None:
    cv=LeaveOneOut()

if param_grid is not None and decoders is not None:
    raise Exception('values for param_grid and decoders both passed, but param_grid is ignored if decoders is passed. choose one.')

#if the groups we're using are actually the same.
if (testset_X is None) and (testset_y is None):
    testset_X = trainset_X
    testset_y = trainset_y
    testset_groups = trainset_groups
    print('Groups are the same.')

results_by_trainset_item = pd.DataFrame({
    'y': trainset_y,
    'group':trainset_groups,
    'y_pred':np.repeat(None,len(trainset_y))#,
    #'y_match':np.repeat(None,len(trainset_y))#just for debugging. delete.
})

Groups are the same.


In [23]:
groups_array = np.array(list(set(testset_groups)))
assert(set(trainset_groups)==set(testset_groups))

if decoders is None:
    #the CV that the inner decoder uses
    cv_inner = GroupKFold(3)
    decoders = [DecoderRegressor(standardize= True,param_grid=param_grid,cv=cv_inner,scoring="r2",
                                  n_jobs=cpus_to_use)]
    print('using default decoder, DecoderRegressor',end='. ')

#we actually use KFold on the group names themselves, then filter across that
#that's equivalent to doing a GroupedKFold on the data.
test_scores = []
results = []

if type(cv)==type(LeaveOneOut()):
    outer_n=len(groups_array)
else:
    outer_n = cv.get_n_splits()
for i, x in enumerate(cv.split(groups_array)):
    train_i = x[0]
    test_i = x[1]
    print("fold " + str(i+1) + " of " + str(outer_n))

    fold_i_results = {}
    train_group_items, test_group_items = groups_array[train_i], groups_array[test_i]
    print('In order to test on a training group of ' +
          str(len(train_group_items)) + ' items, holding out the following subjects:' +
          str(test_group_items),end='. ')
#         print(
#             'held out ' + str(len(test_group_items)) + ' items and trained on ' + str(len(train_group_items)) + ' items',
#             end='. ')

    print('prepping fold data...',end='. ')
    #select training data from the averages
    #print('selecting training data',end='. ')
    train_selector = [i for i, x in enumerate(trainset_groups) if x in train_group_items]
    train_y = trainset_y[train_selector]
    train_X = nib.funcs.concat_images([trainset_X.slicer[...,s] for s in train_selector])
    train_groups = trainset_groups[train_selector]
    #print(train_X.shape,end='. ')
    #print(asizeof_fmt(train_X),end='. ')

    #select testing data from the individual values
    #print('selecting test data',end='. ')
    test_selector = [i for i, x in enumerate(testset_groups) if x in test_group_items]
    test_y = testset_y[test_selector]
    test_X = nib.funcs.concat_images([testset_X.slicer[...,s] for s in test_selector])
    test_groups = testset_groups[test_selector]
    #print(asizeof_fmt(test_X),end='. ')
    #print(test_X.shape,end='. ')


    print("fitting...",end='. ')
    print(asizeof_fmt(train_X),end='. ')

    val_scores = []
    #iterate through decoder objects.
    #this is my way of doing cross-validation across different decoders...
    hyper_scores = []
    train_results = {}
    inner_cv_results = {}
    for r_i, reg in enumerate(decoders):
        cur_r_results = {}
        print('trying decoder ' + str(r_i+1) + ' of ' + str(len(decoders)),end='. ')
        #if there is nested CV within this function the best hyper-paramters are already being chosen
        #we need only to finish the job by identifying the best overall decoder, as the final hyper-parameter
        reg.fit(y=train_y,X=train_X,groups=train_groups)
        print("predicting",end='. ')

        hyper_score = np.max([np.mean(param_values) for param_name, param_values in reg.cv_scores_.items()])
        #think there is a bug here. we should not have to be guessing/ignoring param names.
        #need to report this.

        hyper_scores = hyper_scores + [hyper_score]

        cur_r_results['hyper_score'] = hyper_score
        cur_r_results['cv_scores_'] = reg.cv_scores_
        cur_r_results['cv_params_'] = reg.cv_params_
        inner_cv_results[str(reg)] = cur_r_results

    fold_i_results['train_results']= inner_cv_results

    #identify which was the best
    best_hyper_decoder = decoders[np.where([h==np.max(hyper_scores) for h in hyper_scores])[0][0]]

    #now run JUST that one on this fold.


    #now predict on our test split
    test_score = best_hyper_decoder.score(test_X,test_y)
    test_y_pred = best_hyper_decoder.predict(test_X)
    fold_test_rawdata = pd.DataFrame({
        'y_obs':test_y,
        'y_pred':test_y_pred,
        'y_groups':test_groups

    })
    #results_by_trainset_item.loc[train_selector,'y_pred']
    results_by_trainset_item.loc[test_selector,'y_pred'] = test_y_pred
    #results_by_trainset_item.loc[test_selector,'y_match'] = test_y
    fold_i_results['fold_test_rawdata'] = fold_test_rawdata
    #so we can do scoring externally to this function.

    test_scores = test_scores+[test_score]
    print('test score was:',end='. ')
    print(test_score)

    results = results + [fold_i_results]

    del test_X
    del train_X
    gc.collect() #clean up. this is big data we're working with

#return(test_scores,results,results_by_trainset_item)


fold 1 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV041']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. predicting. test score was:. 1.0
fold 2 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV029']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.75
fold 3 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV022']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.8
fold 4 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV057']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

...

predicting. test score was:. 0.75
fold 39 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV009']. prepping fold data.... fitting.... 1.5 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 1.0
fold 40 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV023']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.8


#### Initial results

In [24]:
np.mean(test_scores)

0.7775000000000001

This is the average train-test performance over 40 subjects LOOCV. Not bad!

In [25]:
test_predictions = pd.concat([testsub['fold_test_rawdata'] for testsub in results])

In [26]:
test_predictions['y_obs']

123      CorrectGo
124    CorrectStop
125     FailedStop
126            Cue
91       CorrectGo
          ...     
64       CorrectGo
65     CorrectStop
66      FailedStop
67             Cue
68        FailedGo
Name: y_obs, Length: 183, dtype: object

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
#accuracy by group
for group in np.unique(test_predictions['y_obs']):
    print(group)
    predictions_for_group = test_predictions.loc[test_predictions['y_obs']==group]
    print(accuracy_score(predictions_for_group['y_pred'],predictions_for_group['y_obs']))

CorrectGo
0.9
CorrectStop
0.7567567567567568
Cue
0.975
FailedGo
0.038461538461538464
FailedStop
0.9


It's important to remember that, during classification, these groups are more-or-less balanced (OK, _much_ fewer FailedGo, but not an order of magnitude less). The rason Failed Go doesn't perform very well must be because we don't have a very good beta of it because there are so few instances of it.

In [30]:
test_predictions['y_obs'].value_counts()

FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: y_obs, dtype: int64

So what next?

So what next?

So what next? Need a way to get at inhibition...

### Next step: wrap this in an encapsulated function...

We didn't actually change the code at all.

In [23]:
#set vars for the function

train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
response_transform_func=condition_resp_trans_func
mask=None

results_filepath=data_path + "SST/train_test_results_" + dataset_name + "_10subs.pkl"

In [24]:
def decoderConstructor(*args, **kwargs):
    return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [56]:
apply_loocv_and_save(
    results_filepath = results_filepath,
    brain_data_filepath = brain_data_filepath,
    train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv",
    subjs_to_use = 40,
    decoderConstructor = decoderConstructor,
    response_transform_func=condition_resp_trans_func,
    mask=None                               
    
)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 40 subjects
starting LeaveOneOut
finished preprocessing
Groups are the same.
fold 1 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV041']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. predicting. test score was:. 1.0
fold 2 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV029']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.75
fold 3 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV022']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

...

predicting. test score was:. 1.0
fold 40 of 40
In order to test on a training group of 39 items, holding out the following subjects:['DEV023']. prepping fold data.... fitting.... 1.4 GiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.8
1.0
1.0
running one more time on whole dataset for beta map


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


finished learning
saved.


In [57]:
metadata = pickle.load(open(results_filepath,'rb'))


In [58]:
test_predictions = pd.concat([testsub['fold_test_rawdata'] for testsub in metadata[1]])
#accuracy by group
for group in np.unique(test_predictions['y_obs']):
    print(group)
    predictions_for_group = test_predictions.loc[test_predictions['y_obs']==group]
    print(accuracy_score(predictions_for_group['y_pred'],predictions_for_group['y_obs']))

CorrectGo
0.9
CorrectStop
0.7567567567567568
Cue
0.975
FailedGo
0.038461538461538464
FailedStop
0.9


That looks good. Now let's try with an abridged dataset with just CorrectStop and FailedStop.

In [60]:
test_predictions['y_obs'].value_counts()

FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: y_obs, dtype: int64

Might make more sense to do CorrectStop vs. CorrectGo

### Thoughts with Meeting with Ell

Think about looking at the image map and mask out motor cortex or mask-in only PFC

To implement plan from https://uoregonctn.atlassian.net/wiki/spaces/DE/pages/1730019339/TESQ-E+and+measures+of+response+inhibition+and+suppression:
        

- Do a regression of a simple CorrectStop vs. CorrectGo on subject-level SSRT

#### proposal

Two class (e.g., CorrectStop vs. CorrectGo)

Leave-One-Out classifier...

Neural similarity of a subject to the group with a Stop vs. Go contrast might actually be indicative of performance.

#### proposal

Building the measure: Dissimilarity measure of CorrectStop vs. CorrectGo within subject. Repeat for each subject and we have dissimilarity measures across the group.

Ultimate research question: does this better predict FFQ?


#### proposal

within-subject classification of trials (CorrectStop vs. CorrectGo) usign LOOCV WITHIN each subject/run

Then use prediction accuracy or disimilarity as an indivdual difference

Enhance with focus on specific theoretically relevant region, or make it harder using CorrectStop vs. FailedStop ???

#### Follow-up

Might be lots of other ideas but these would be a good start.
Would be cool to have a correlation matrix of all of these things together.

## Post-error slowing
Subdivide among stop trials, an instead o correct vs. incorrect, split them as previous stop ERROR vs. previous stop NO ERROR.

Then might try within-subject classification of trials.

## what's the wider scientific value of this which might guide our choices?

Not predicting FFQ in itself but "construct discovery"--how does behavior on a simple task like this and how does neural activity on a task like this influence FFQ and ultimately health outcomes?



##  ...and classify FailedStop vs. CorrectStop

In [10]:
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_58subs_correct_cond.pkl'

In [11]:
#set vars for the function
train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
response_transform_func=condition_resp_trans_func
mask=None

results_filepath=data_path + "SST/train_test_results_" + dataset_name + "_40subs_twoclasses.pkl"

def decoderConstructor(*args, **kwargs):
    return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [12]:
apply_loocv_and_save(
    results_filepath = results_filepath,
    brain_data_filepath = brain_data_filepath,
    train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv",
    subjs_to_use = 58,
    decoderConstructor = decoderConstructor,
    response_transform_func=condition_resp_trans_func,
    mask=None
)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 55 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 152
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


110
110
using 58 subjects
starting LeaveOneOut
finished preprocessing
Groups are the same.
fold 1 of 55
In order to test on a training group of 54 items, holding out the following subjects:['DEV057']. prepping fold data.... fitting.... 891.6 MiB. trying decoder 1 of 1. predicting. test score was:. 1.0
fold 2 of 55
In order to test on a training group of 54 items, holding out the following subjects:['DEV058']. prepping fold data.... fitting.... 891.6 MiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 1.0
fold 3 of 55
In order to test on a training group of 54 items, holding out the following subjects:['DEV062']. prepping fold data.... fitting.... 891.6 MiB. trying decoder 1 of 1. 

...

predicting. test score was:. 1.0
fold 55 of 55
In order to test on a training group of 54 items, holding out the following subjects:['DEV049']. prepping fold data.... fitting.... 891.6 MiB. trying decoder 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 1.0
1.0
1.0
running one more time on whole dataset for beta map


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


finished learning
saved.


In [11]:
results_2c = pickle.load(open(results_filepath,'rb'))

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
test_predictions = pd.concat([testsub['fold_test_rawdata'] for testsub in results_2c[1]])
#accuracy by group
for group in np.unique(test_predictions['y_obs']):
    print(group)
    predictions_for_group = test_predictions.loc[test_predictions['y_obs']==group]
    print(accuracy_score(predictions_for_group['y_pred'],predictions_for_group['y_obs']))

CorrectGo
0.95
CorrectStop
0.95


In [14]:
def sbg(x):
    return(accuracy_score(x['y_pred'],x['y_obs']))


test_predictions.groupby('y_groups').apply(func=sbg)

y_groups
DEV005    0.0
DEV006    1.0
DEV010    1.0
DEV013    1.0
DEV014    1.0
DEV015    1.0
DEV016    1.0
DEV017    1.0
DEV018    1.0
DEV019    1.0
DEV021    1.0
DEV022    1.0
DEV023    1.0
DEV024    1.0
DEV026    1.0
DEV027    1.0
DEV028    1.0
DEV029    1.0
DEV030    1.0
DEV034    1.0
DEV035    1.0
DEV036    1.0
DEV039    1.0
DEV040    1.0
DEV041    1.0
DEV042    1.0
DEV043    1.0
DEV046    1.0
DEV048    1.0
DEV049    1.0
DEV050    1.0
DEV051    1.0
DEV052    1.0
DEV053    1.0
DEV055    0.0
DEV056    1.0
DEV057    1.0
DEV058    1.0
DEV059    1.0
DEV060    1.0
dtype: float64

Well, that performance is very good! Almost too good, but we have done everything right here--I am pretty confident we've managed the data properly to avoid overfitting. The only potential problem is that we do cross-validation in the inner CV, built into the decoder, without properly handling how that went, but ~~it shouldn't make a big difference~~ I don't think that's a problem at all. In the end, it still has to then make a prediction to new data that has not been optimized for.

Next steps is to proceed with some of the bespoke similarity analysis I was going to do.

Since we're here, though, do we want to try a simple regression against some subject-level variables? See `SST_against_inddiv.ipynb`